In [1]:
# imports, setup
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_validate, GridSearchCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer

import matplotlib.pyplot as plt
import seaborn as sns

# load data
df = pd.read_csv("steel.csv")

# features/target
X = df.drop("tensile_strength", axis=1)
y = df["tensile_strength"]

scoring = {
    "R2": make_scorer(r2_score),
    "MAE": make_scorer(mean_absolute_error)
}

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#SVR

In [ ]:
#KNNR